# Goal 1: Following example from Keras demo

In [17]:
from __future__ import print_function
import numpy as np
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras import backend as K
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV  
import pickle
import seaborn as sns; sns.set()
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import os

In [3]:
'''Train a simple deep CNN on the CIFAR10 small images dataset.
It gets to 75% validation accuracy in 25 epochs, and 79% after 50 epochs.
(it's still underfitting at that point, though).
'''

np.random.seed(9)

batch_size = 32
num_classes = 10
epochs = 25
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

# The data, shuffled and split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4)

# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
Using real-time data augmentation.
Epoch 1/25
1563/1563 [==============================] - 232s 148ms/step - loss: 1.8719 - acc: 0.3149 - val_loss: 1.5580 - val_acc: 0.4327
Epoch 2/25
1563/1563 [==============================] - 234s 150ms/step - loss: 1.5567 - acc: 0.4320 - val_loss: 1.3440 - val_acc: 0.5179
Epoch 3/25
1563/1563 [==============================] - 241s 154ms/step - loss: 1.4341 - acc: 0.4843 - val_loss: 1.2413 - val_acc: 0.5606
Epoch 4/25
1563/1563 [==============================] - 238s 152ms/step - loss: 1.3361 - acc: 0.5220 - val_loss: 1.2029 - val_acc: 0.5717
Epoch 5/25
1563/1563 [==============================] - 238s 152ms/step - loss: 1.2630 - acc: 0.5495 - val_loss: 1.1410 - val_acc: 0.5978
Epoch 6/25
1563/1563 [==============================] - 234s 150ms/step - loss: 1.1982 - acc: 0.5729 - val_loss: 1.0290 - val_acc: 0.6416
Epoch 7/25
1563/1563 [==============================] - 233s 149

# Question 2: Improving the accuracy beyond the example

In [15]:
# Define the model
model = Sequential()
model.add(Conv2D(64, (3, 3), padding='same', activation='relu', input_shape=x_train.shape[1:]))
model.add(BatchNormalization(axis=-1))
model.add(Conv2D(48, (3, 3), activation='relu'))
model.add(BatchNormalization(axis=-1))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.20))
model.add(Conv2D(96, (3, 3), activation='relu', padding='same'))
model.add(BatchNormalization(axis=-1))
model.add(Conv2D(96, (3, 3), activation='relu'))
model.add(BatchNormalization(axis=-1))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(192, (3, 3), activation='relu', padding='same'))
model.add(BatchNormalization(axis=-1))
model.add(Conv2D(192, (3, 3), activation='relu'))
model.add(BatchNormalization(axis=-1))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.20))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization(axis=-1))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))


model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)
    print('Beginning model.')
    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_57 (Conv2D)           (None, 32, 32, 64)        1792      
_________________________________________________________________
batch_normalization_54 (Batc (None, 32, 32, 64)        256       
_________________________________________________________________
conv2d_58 (Conv2D)           (None, 30, 30, 48)        27696     
_________________________________________________________________
batch_normalization_55 (Batc (None, 30, 30, 48)        192       
_________________________________________________________________
max_pooling2d_29 (MaxPooling (None, 15, 15, 48)        0         
_________________________________________________________________
dropout_40 (Dropout)         (None, 15, 15, 48)        0         
_________________________________________________________________
conv2d_59 (Conv2D)           (None, 15, 15, 96)        41568     
__________

## Confirming accuracy

In [21]:
# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 [==============================] - 124s 12ms/step
Test loss: 0.375709511614
Test accuracy: 0.8753


### Saving data

In [19]:
#Saving data
with open( "CIFAR10.pickle", "wb" ) as file:
    pickle.dump(( intermediate_output, intermediate_output_test), file)
#Loading data
with open( "CIFAR10.pickle", "rb" ) as file:
    (intermediate_output, intermediate_output_test) = pickle.load(file)

# Goal 3: Mofifying the learning rate

In [25]:
# define learning rate function

from keras.optimizers import SGD
from keras.callbacks import LearningRateScheduler

class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = []

    def on_epoch_end(self, epoch, logs={}):
        self.losses.append(logs.get('loss'))
        #sd.append(step_decay(len(self.losses)))
        print('       Learning rate: ', K.eval(self.model.optimizer.lr))
        #print('       Loss Check: ', self.losses[-1])


def scheduler(epoch):
    if epoch > 0:
        eta0 = 0.001/np.exp(1.8173) #Using the previous model's initial loss, 0.001 is the default lr vlue for Adam.
        eta1 = eta0*np.exp(history.losses[-1])
    else:
        eta1 = 0.001
    return eta1

print('Compiling model {}.'.format('model 2'))

model_2 = Sequential()
model_2.add(Conv2D(64, (3, 3), padding='same', activation='relu', input_shape=x_train.shape[1:]))
model_2.add(BatchNormalization(axis=-1))
model_2.add(Conv2D(48, (3, 3), activation='relu'))
model_2.add(BatchNormalization(axis=-1))
model_2.add(MaxPooling2D(pool_size=(2, 2)))
model_2.add(Dropout(0.20))
model_2.add(Conv2D(96, (3, 3), activation='relu', padding='same'))
model_2.add(BatchNormalization(axis=-1))
model_2.add(Conv2D(96, (3, 3), activation='relu'))
model_2.add(BatchNormalization(axis=-1))
model_2.add(MaxPooling2D(pool_size=(2, 2)))
model_2.add(Dropout(0.25))
model_2.add(Conv2D(192, (3, 3), activation='relu', padding='same'))
model_2.add(BatchNormalization(axis=-1))
model_2.add(Conv2D(192, (3, 3), activation='relu'))
model_2.add(BatchNormalization(axis=-1))
model_2.add(MaxPooling2D(pool_size=(2, 2)))
model_2.add(Dropout(0.20))
model_2.add(Flatten())
model_2.add(Dense(512, activation='relu'))
model_2.add(BatchNormalization(axis=-1))
model_2.add(Dropout(0.5))
model_2.add(Dense(256, activation='relu'))
model_2.add(Dropout(0.5))
model_2.add(Dense(num_classes, activation='softmax'))
model_2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


if not data_augmentation:
    print('Not using data augmentation.')
    model_2.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)
    print('Beginning model.')
    # Fit the model on the batches generated by datagen.flow().
    lrate=LearningRateScheduler(scheduler)   
    history = LossHistory()
    history_model2 = model_2.fit(x_train, y_train, batch_size=batch_size, epochs=25, verbose=1, \
                             validation_data=(x_test, y_test), callbacks=[history, lrate])

#print(history.losses)

Compiling model model 2.
Using real-time data augmentation.
Beginning model.
Train on 50000 samples, validate on 10000 samples
Epoch 1/25
50000/50000 [==============================] - 963s 19ms/step - loss: 1.7920 - acc: 0.4039 - val_loss: 1.3478 - val_acc: 0.5329
Epoch 2/25
50000/50000 [==============================] - 876s 18ms/step - loss: 1.1141 - acc: 0.6103 - val_loss: 0.9388 - val_acc: 0.6746
Epoch 3/25
50000/50000 [==============================] - 860s 17ms/step - loss: 0.8493 - acc: 0.7089 - val_loss: 0.7255 - val_acc: 0.7477
Epoch 4/25
50000/50000 [==============================] - 878s 18ms/step - loss: 0.7169 - acc: 0.7546 - val_loss: 0.6345 - val_acc: 0.7783
Epoch 5/25
50000/50000 [==============================] - 882s 18ms/step - loss: 0.6235 - acc: 0.7889 - val_loss: 0.6047 - val_acc: 0.7909
Epoch 6/25
50000/50000 [==============================] - 888s 18ms/step - loss: 0.5670 - acc: 0.8064 - val_loss: 0.5196 - val_acc: 0.8197
Epoch 7/25
50000/50000 [===============

In [28]:
# Score trained model.
scores = model_2.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 [==============================] - 61s 6ms/step
Test loss: 0.535131616455
Test accuracy: 0.852
